This notebook is a proto for the ETL process which will be complete in etl.py

In [1]:
import psycopg2
import os
import glob
import pandas as pd
from sql_queries import * 

In [2]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=admin")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)
conn.set_session(autocommit=True)

In [3]:
# the following funtion lists all files in a given filepath
def get_files(filepath):

    all_files=[]
    for root, dirs, files in os.walk(filepath):
        files=glob.glob(os.path.join(root,'*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))
    return (all_files)
    


### Process song_data to extract songs & artists dimensional tables
- In this notebook, for the sake of demo, I'll perform ETL on a single song file and load a single record into each table to start.
- I'll use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- I'll select the first song in that list
- I'll read the song file and view the data

In [4]:
song_files=get_files("data/song_data")

In [5]:
filepath=song_files[0]

In [6]:
df_songs=pd.read_json(filepath, lines=True)

In [7]:
df_songs

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


#### Songs Table 

Select the columns needed to populate songs table

In [8]:
songs_df=df_songs[["song_id", "title", "artist_id", "year", "duration"]].values[0]

In [9]:
songs_data=songs_df.tolist()

In [10]:
songs_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

Implement the `song_table_insert` in `sql_queries.py` to insert the above record into `songs` table.
Remember to run create_tables.py before running the cell below to ensure you have created/resetted the `songs` table in `sparkifydb` database

In [11]:
cur.execute(song_table_insert, songs_data)

#### Artists table

Select the columns needed to populate artists table

In [12]:
artists_df=df_songs[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values[0]

In [13]:
artists_data=artists_df.tolist()

In [14]:
cur.execute(artist_table_insert, artists_data)

### Process log_data to extract  time & users dimensional tables
- In this notebook, for the sake of demo, I'll perform ETL on a single log file and load it into each table to start.
- I'll use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`

In [15]:
log_files=get_files("data/log_data")
log_files[0]

'/Users/omarfessi/Desktop/data-modeling-Postgres/data/log_data/2018/11/2018-11-11-events.json'

In [16]:
df_logs=pd.read_json(log_files[0], lines=True)
df_logs.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


#### Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime ( because the timestamp is in milliseconds 
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [17]:
df_logs=df_logs[df_logs['page']=='NextSong']

In [18]:
df_logs

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Sidewalk Prophets,Logged In,Molly,F,0,Taylor,260.62322,free,"St. Louis, MO-IL",PUT,NextSong,1.540993e+12,464,You Love Me Anyway (Album),200,1541970568796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",35
91,Rise Against,Logged In,Molly,F,1,Taylor,221.17832,free,"St. Louis, MO-IL",PUT,NextSong,1.540993e+12,464,Torches,200,1541970828796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",35
92,K'Naan,Logged In,Molly,F,2,Taylor,220.49914,free,"St. Louis, MO-IL",PUT,NextSong,1.540993e+12,464,Wavin' Flag,200,1541971049796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",35
93,Patrick Jumpen,Logged In,Ryan,M,0,Smith,208.87465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,480,Holiday,200,1541979540796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


In [19]:
t=pd.to_datetime(df_logs['ts'],unit='ms')
time_data=[t,\
          t.dt.hour,\
          t.dt.day,\
          t.dt.week,\
          t.dt.month,\
          t.dt.year,\
          t.dt.dayofweek]
column_labels=['ts', 'hour', 'day', 'week', 'month', 'year', 'dayofweek']
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()

,ts,hour,day,week,month,year,dayofweek
0,2018-11-11 02:33:56.796,2,11,45,11,2018,6
1,2018-11-11 02:36:10.796,2,11,45,11,2018,6
2,2018-11-11 02:40:34.796,2,11,45,11,2018,6
4,2018-11-11 04:36:13.796,4,11,45,11,2018,6
5,2018-11-11 04:36:46.796,4,11,45,11,2018,6


##### Insert Records row by row into Time table
I implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [20]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, row.tolist())

#### users table

In [21]:
users_df=df_logs[["userId", "firstName", "lastName", "gender", "level"]]

In [22]:
for i, row in users_df.iterrows():
    cur.execute(user_table_insert, row.tolist())

#### songplays Table
##### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

##### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [23]:
for i, row in df_logs.iterrows():
    cur.execute(song_select, (row.song, row.artist, row.length))
    results=cur.fetchone()
    
    if results:
        song_id, artist_id = results
    else :
        song_id, artist_id = None, None
    
    songplay_data=(pd.to_datetime(row.ts, unit='ms'),\
                   row.userId,\
                   song_id,\
                   artist_id,\
                   row.level,\
                   row.sessionId,\
                   row.location,\
                   row.userAgent)
    
    cur.execute(songplay_table_insert, songplay_data)                

### Close Connection to Sparkify Database

In [24]:
conn.close()